<a href="https://colab.research.google.com/github/Talaat-mostafa938/Data-Analysis/blob/main/Garbage_Calssification_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
from PIL import Image

import tensorflow as tf
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , Dropout , GlobalAveragePooling2D , BatchNormalization
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [7]:
od.download('https://www.kaggle.com/datasets/mostafaabla/garbage-classification')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: talaatmostafa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mostafaabla/garbage-classification


100%|██████████| 239M/239M [00:00<00:00, 750MB/s] 

In [8]:
path = '/content/garbage-classification/garbage_classification'

In [9]:
img_size = 128
batch_size = 32

In [10]:
filepaths = []
labels = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            filepath = os.path.join(root, file)
            filepaths.append(filepath)
            label = os.path.basename(root)
            labels.append(label)

data_df = pd.DataFrame({'filepath': filepaths, 'original_label': labels})


def unify_glass_labels(label):
    if 'glass' in label:
        return 'glass'
    return label

data_df['unified_label'] = data_df['original_label'].apply(unify_glass_labels)
data_df.drop(columns=['original_label'], inplace=True)
data_df.head()

,filepath,unified_label
0,/content/garbage-classification/garbage_classi...,clothes
1,/content/garbage-classification/garbage_classi...,clothes
2,/content/garbage-classification/garbage_classi...,clothes
3,/content/garbage-classification/garbage_classi...,clothes
4,/content/garbage-classification/garbage_classi...,clothes


In [11]:
train_df, test_df = train_test_split(
    data_df,
    test_size=0.2,
    stratify=data_df['unified_label'],
    random_state=42
)

In [12]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
)

train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='unified_label',
    target_size = (img_size , img_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
)

test_data = test_gen.flow_from_dataframe(
    test_df,
    x_col='filepath',
    y_col='unified_label',
    target_size = (img_size , img_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = False
)


Found 12412 validated image filenames belonging to 10 classes.
Found 3103 validated image filenames belonging to 10 classes.


In [13]:
classes = train_data.class_indices
classes

{'battery': 0,
 'biological': 1,
 'cardboard': 2,
 'clothes': 3,
 'glass': 4,
 'metal': 5,
 'paper': 6,
 'plastic': 7,
 'shoes': 8,
 'trash': 9}

In [14]:
length =  len(classes)
length

10

In [15]:
model_1 = Sequential([
    Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(256, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(length , activation = 'softmax')
])

model_1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           51

 Total params: 1,250,506 (4.77 MB)

 Trainable params: 1,247,946 (4.76 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [16]:
from tensorflow.keras.optimizers import Adam

model_1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

epchos = 10
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model_1.fit(
    train_data,
    validation_data=test_data,
    epochs=epchos,
    callbacks=[early_stopping]
)

Epoch 1/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 118s 244ms/step - accuracy: 0.3998 - loss: 2.0508 - val_accuracy: 0.3780 - val_loss: 2.0261
Epoch 2/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 73s 189ms/step - accuracy: 0.5322 - loss: 1.4557 - val_accuracy: 0.3735 - val_loss: 1.9413
Epoch 3/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 73s 188ms/step - accuracy: 0.5689 - loss: 1.2800 - val_accuracy: 0.5730 - val_loss: 1.2519
Epoch 4/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.5889 - loss: 1.2237 - val_accuracy: 0.4360 - val_loss: 2.1222
Epoch 5/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 75s 192ms/step - accuracy: 0.6095 - loss: 1.1610 - val_accuracy: 0.5833 - val_loss: 1.3367
Epoch 6/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 73s 188ms/step - accuracy: 0.6236 - loss: 1.1160 - val_accuracy: 0.5485 - val_loss: 1.3284
Epoch 7/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.6393 - loss: 1.0757 - val_accuracy: 0.5588 - val_loss: 1.5304
Epoch 8/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.6458 - loss: 

In [18]:
loss , accuarcy = model_1.evaluate(test_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuarcy}")

97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.5760 - loss: 1.2427
Test Loss: 1.2519035339355469, Test Accuracy: 0.5729938745498657


In [19]:
y_pred_cnn = model_1.predict(test_data)
y_pred_cnn = np.argmax(y_pred_cnn, axis=1)

97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step


In [26]:
base_model = tf.keras.applications.DenseNet121(input_shape=(img_size , img_size , 3) ,include_top=False,weights='imagenet')
base_model.trainable = True

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(length , activation='softmax')(x)
model_DenseNet121 = Model(inputs=base_model.input, outputs=predictions)

model_DenseNet121.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 134, 134,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 32, 32,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 32, 32,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 32, 32,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 32, 32,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 32, 32,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 32, 32,    │     12,288 │ conv2_block2_0_r

 Total params: 7,182,218 (27.40 MB)

 Trainable params: 7,096,266 (27.07 MB)

 Non-trainable params: 85,952 (335.75 KB)

In [27]:
model_DenseNet121.compile(loss ='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
epchos = 20
earlystop = EarlyStopping(patience=5, restore_best_weights=True , monitor='val_loss')

history = model_DenseNet121.fit(
    train_data,
    validation_data=test_data,
    epochs=epchos,
    callbacks=[earlystop]
)

Epoch 1/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 292s 415ms/step - accuracy: 0.6714 - loss: 1.0248 - val_accuracy: 0.6084 - val_loss: 1.4988
Epoch 2/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 77s 198ms/step - accuracy: 0.8101 - loss: 0.5779 - val_accuracy: 0.7944 - val_loss: 0.6624
Epoch 3/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 79s 204ms/step - accuracy: 0.8447 - loss: 0.4666 - val_accuracy: 0.7193 - val_loss: 0.9210
Epoch 4/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 77s 199ms/step - accuracy: 0.8582 - loss: 0.4287 - val_accuracy: 0.8540 - val_loss: 0.4706
Epoch 5/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 77s 197ms/step - accuracy: 0.8704 - loss: 0.3900 - val_accuracy: 0.7000 - val_loss: 1.0363
Epoch 6/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 77s 198ms/step - accuracy: 0.8801 - loss: 0.3571 - val_accuracy: 0.8392 - val_loss: 0.5069
Epoch 7/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 82s 198ms/step - accuracy: 0.8970 - loss: 0.3160 - val_accuracy: 0.8034 - val_loss: 0.6037
Epoch 8/20
388/388 ━━━━━━━━━━━━━━━━━━━━ 76s 196ms/step - accuracy: 0.9005 - loss: 

In [29]:
loss , accuracy = model_DenseNet121.evaluate(test_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.8975 - loss: 0.3198
Test Loss: 0.3416198194026947, Test Accuracy: 0.8904286026954651
